# Ray RLlib Multi-Armed Bandits - Market Bandit Example

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

Now that we've learned about multi-armed bandits and methods for optimizing rewards, let's look at real-world applications, starting with a stock market example. We'll also learn a little more about configuring RLlib trainers.

We'll load a dataset derived from this [NYU Stern table](http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html) that shows returns for nearly a century of market data, including dividends and adjustments for inflation. The `market.tsv` file in this folder contains the data.

In [1]:
import pandas as pd
import numpy as np
import os, sys

In [2]:
# Some properties we'll need:
DEFAULT_MAX_INFLATION = 100.0
DEFAULT_TICKERS = ["sp500", "t.bill", "t.bond", "corp"]
DEFAULT_DATA_FILE = os.path.abspath(os.path.curdir) + '/market.tsv'  # full path

In [3]:
def load_market_data (file_name):
    with open(file_name, "r") as f:
        return pd.read_table(f)

In [4]:
df = load_market_data(DEFAULT_DATA_FILE)
df

,year,inflation,sp500,t.bill,t.bond,corp
0,1928,-1.15,45.49,4.28,2.01,4.42
1,1929,0.00,-8.30,3.16,4.20,3.02
2,1930,-2.67,-23.07,7.42,7.41,3.30
3,1931,-8.93,-38.33,12.34,7.00,-7.41
4,1932,-10.30,1.85,12.68,21.28,37.78
...,...,...,...,...,...,...
87,2015,0.12,1.26,-0.07,1.16,-0.82
88,2016,1.26,10.38,-0.93,-0.56,8.99
89,2017,2.13,19.07,-1.17,0.66,7.44
90,2018,2.44,-6.51,-0.49,-2.40,-5.08


As you can see the data spans 92 years, from 1928 to 2019. The columns represent:
  * the year
  * inflation rate at the time
  * [S&P500](https://en.wikipedia.org/wiki/S%26P_500_Index) (composite stock index)
  * [Treasury Bills](https://www.investopedia.com/terms/t/treasurybill.asp) (short-term gov bonds)
  * [Treasury Bonds](https://www.investopedia.com/terms/t/treasurybond.asp) (long-term gov bonds)
  * [Moody's Baa Corporate Bonds](https://en.wikipedia.org/wiki/Moody%27s_Investors_Service#Moody's_credit_ratings) (moderate risk)

In [5]:
df.describe()

,year,inflation,sp500,t.bill,t.bond,corp
count,92.00000,92.000000,92.000000,92.000000,92.000000,92.000000
mean,1973.50000,3.041957,8.413261,0.434239,2.166413,4.216630
std,26.70206,3.803579,19.619605,3.573035,8.126432,8.625809
min,1928.00000,-10.300000,-38.900000,-12.050000,-14.570000,-14.850000
25%,1950.75000,1.415000,-2.740000,-1.185000,-2.620000,-1.322500
50%,1973.50000,2.750000,10.515000,0.590000,1.070000,3.910000
75%,1996.25000,4.275000,20.622500,2.117500,7.037500,9.287500
max,2019.00000,14.390000,58.200000,12.680000,25.140000,37.780000


"Corp" refers to corporate bonds.

## Analysis of the Data

What are the worst case and best case scenarios? In other words, if one could predict the future market performance, what are the possible ranges of total failure vs. total success over the past century? By "total", we mean what if you had all your money in a given year invested in the worst performing _sector_ (S&P500, T bills, or other) or you were invested in the best performing sector for that year.

In [10]:
n_years = len(df)
min_list = []
max_list = []

for i in range(n_years):
    row = df.iloc[i, 2:]
    min_list.append(min(row))
    max_list.append(max(row))
    
print("{:5.2f}% worst case annualized".format(sum(min_list) / n_years))
print("{:5.2f}% best case annualized".format(sum(max_list) / n_years))

-5.64% worst case annualized
15.18% best case annualized


In [11]:
sys.path.append('../..')
from util.line_plots import plot_line, plot_line_with_stddev, plot_between_lines
from bokeh_util import plot_cumulative_regret

In [12]:
import bokeh
bokeh.io.reset_output()
bokeh.io.output_notebook()

Loading BokehJS ...

In [13]:
min_max = pd.DataFrame.from_dict({'year': df['year'], 'min':min_list, 'max':max_list})
min_max

,year,min,max
0,1928,2.01,45.49
1,1929,-8.30,4.20
2,1930,-23.07,7.42
3,1931,-38.33,12.34
4,1932,1.85,37.78
...,...,...,...
87,2015,-0.82,1.26
88,2016,-0.93,10.38
89,2017,-1.17,19.07
90,2018,-6.51,-0.49


In [14]:
plot_between_lines(min_max, x_col='year', lower_col='min', upper_col='max', 
                   title='Best to Worst', x_axis_label='year', y_axis_label='%')

From the graph, there are some years where the performance varies widely, while other years everything returns about the same performance.

## Defining an Environment

Now let's define a Gym environment so that we can train a contextual bandit to optimize annual investments over that period.

In [15]:
import gym
from gym.spaces import Discrete, Box
from gym.utils import seeding
import numpy as np
import random

This is the bandit we'll use to represent the market "environment".

In [16]:
class MarketBandit (gym.Env):
    
    def __init__ (self, config={}):
        self.max_inflation = config.get('max-inflation', DEFAULT_MAX_INFLATION)
        self.tickers = config.get('tickers', DEFAULT_TICKERS)
        self.data_file = config.get('data-file', DEFAULT_DATA_FILE)
        print(f"MarketBandit: max_inflation: {self.max_inflation}, tickers: {self.tickers}, data file: {self.data_file} (config: {config})")

        self.action_space = Discrete(4)
        self.observation_space = Box(
            low  = -self.max_inflation,
            high =  self.max_inflation,
            shape=(1, )
        )
        self.df = load_market_data(self.data_file)
        self.cur_context = None


    def reset (self):
        self.year = self.df["year"].min()
        self.cur_context = self.df.loc[self.df["year"] == self.year]["inflation"][0]
        self.done = False
        self.info = {}

        return [self.cur_context]


    def step (self, action):
        if self.done:
            reward = 0.
            regret = 0.
        else:
            row = self.df.loc[self.df["year"] == self.year]

            # calculate reward
            ticker = self.tickers[action]
            reward = float(row[ticker])

            # calculate regret
            max_reward = max(map(lambda t: float(row[t]), self.tickers))
            regret = round(max_reward - reward)

            # update the context
            self.cur_context = float(row["inflation"])

            # increment the year
            self.year += 1

            if self.year >= self.df["year"].max():
                self.done = True

        context = [self.cur_context]
        #context = self.observation_space.sample()

        self.info = {
            "regret": regret,
            "year": self.year
        }
         
        return [context, reward, self.done, self.info]


    def seed (self, seed=None):
        """Sets the seed for this env's random number generator(s).
        Note:
            Some environments use multiple pseudorandom number generators.
            We want to capture all such seeds used in order to ensure that
            there aren't accidental correlations between multiple generators.
        Returns:
            list<bigint>: Returns the list of seeds used in this env's random
              number generators. The first value in the list should be the
              "main" seed, or the value which a reproducer should pass to
              'seed'. Often, the main seed equals the provided 'seed', but
              this won't be true if seed=None, for example.
        """
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

Let's see it in action:

In [17]:
bandit = MarketBandit()
bandit.reset()

for i in range(10):
    action = bandit.action_space.sample()
    obs = bandit.step(action)
    print(action, obs)

MarketBandit: max_inflation: 100.0, tickers: ['sp500', 't.bill', 't.bond', 'corp'], data file: /Users/deanwampler/projects/anyscale/academy/academy-git/ray-rllib/multi-armed-bandits/market.tsv (config: {})
2 [[-1.15], 2.01, False, {'regret': 43, 'year': 1929}]
3 [[0.0], 3.02, False, {'regret': 1, 'year': 1930}]
1 [[-2.67], 7.42, False, {'regret': 0, 'year': 1931}]
0 [[-8.93], -38.33, False, {'regret': 51, 'year': 1932}]
0 [[-10.3], 1.85, False, {'regret': 36, 'year': 1933}]
1 [[-5.19], 6.49, False, {'regret': 52, 'year': 1934}]
2 [[3.48], 4.33, False, {'regret': 10, 'year': 1935}]
3 [[2.55], 10.49, False, {'regret': 33, 'year': 1936}]
0 [[1.03], 30.6, False, {'regret': 0, 'year': 1937}]
0 [[3.73], -37.66, False, {'regret': 35, 'year': 1938}]


/Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


We can use this environment in a kind of *monte carlo simulation* to measure a baseline for what the rewards would be over a long period if you always used a random action.

In [18]:
done = 1
reward_list = []
iterations = 10000 #50000

for i in range(iterations):
    if done == 1:
        bandit.reset()

    action = bandit.action_space.sample()
    obs = bandit.step(action)
    context, reward, done, info = obs
    reward_list.append(reward)
    #print(action, context, reward, done, info)

In [19]:
df_mc = pd.DataFrame(reward_list, columns=["reward"])
df_mc.mean()

reward    3.717549
dtype: float64

Depending on the number of iterations, you'll probably get a value approaching 3.75% as a baseline for random actions. That's more than the -5.64% worst case and must less than 15.18% best case for the reward!

In [21]:
plot_line(df_mc, x_col='index', y_col='reward', title='Reward Over Iterations')

([image](../../images/rllib/MarketReward-Random.png))

Yes, it looks quite random... There's no improvement happening at all.

## Training a policy in RLlib

Now let's train a policy using our contextual bandit, specifically using _Linear Thompson Sampling_ in RLlib. Hopefully it will do better than the random results we just computed!

Recall in the `__init__()` method for `MarketBandit` that we set some parameters from the passed in `config` object (with defaults). We don't construct this explicitly ourselves. Rather, RLlib will do this. So, we need to construct the canonical `config` object we want to use. To do this, we use the idioms shown in the next several cells:

In [22]:
from ray.rllib.agents.trainer import with_base_config, with_common_config
from ray.rllib.contrib.bandits.agents.lin_ts import TS_CONFIG
from ray.rllib.contrib.bandits.agents.lin_ts import LinTSTrainer
import ray

Initialize Ray as required:

In [23]:
!../../tools/start-ray.sh --check --verbose

INFO: Ray is already running.


In [24]:
ray.init(address='auto', ignore_reinit_error=True)

2020-06-20 11:44:55,706	WARNING worker.py:809 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.


{'node_ip_address': '192.168.1.149',
 'raylet_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:11135',
 'object_store_address': '/tmp/ray/session_2020-06-19_14-40-43_289473_44550/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-19_14-40-43_289473_44550/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-06-19_14-40-43_289473_44550'}

We need a custom config object with our parameters for `MarketBandit`. We do this building on the default `TS_CONFIG` object for _LinTS_:

In [25]:
market_config = with_base_config(TS_CONFIG, {
    "env":           MarketBandit,
    'max-inflation': DEFAULT_MAX_INFLATION,
    'tickers':       DEFAULT_TICKERS,
    'data-file':     DEFAULT_DATA_FILE
})

stop = {
    "training_iteration": 100
}

Also, we'll define a custom trainer, which builds on the `LinTSTrainer`, with "updates". Note that it's the first argument we'll pass to `tune.run()` in the following cell. When all we need is `LinTSTrainer`, as is, and no extra custom config settings, we can just pass the string `contrib/LinTS` to `tune.run()`.  

In [26]:
MarketLinTSTrainer = LinTSTrainer.with_updates(
    name="MarketLinTSTrainer",
    default_config=market_config,      # Will be merged with Trainer.COMMON_CONFIG (rllib/agent/trainer.py)
    #default_policy=[somePolicyClass]  # If we had a policy...
)

In [27]:
analysis = ray.tune.run(
    MarketLinTSTrainer,
    config=market_config,
    stop=stop,
    num_samples=3,    
    checkpoint_at_end=True,
    verbose=2,              # Change to 0 or 1 to reduce the output.
    ray_auto_init=False,    # Don't allow Tune to initialize Ray.
)

Trial name,status,loc
MarketLinTSTrainer_MarketBandit_00000,RUNNING,
MarketLinTSTrainer_MarketBandit_00001,PENDING,
MarketLinTSTrainer_MarketBandit_00002,PENDING,


(pid=44560) 2020-06-20 11:45:07,800	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=44560) 2020-06-20 11:45:07,803	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=44560) /Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=44560)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=44560) 2020-06-20 11:45:07,816	INFO trainable.py:217 -- Getting current IP.
(pid=44560) 2020-06-20 11:45:07,816	WARNING util.py:37 -- Install gputil for GPU system monitoring.
(pid=44565) MarketBandit: max_inflation: 100.0, tickers: ['sp500', 't.bill', 't.bond', 'corp'], data file: /Users/deanwampler/projects/anyscale/academy/academy-git/ray-rllib/multi-armed-bandits/market.tsv (config: {})
(pid=44565) 2020-06-20 11:45:07,80

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,RUNNING,192.168.1.149:44565,3,0.559135,300,411.91
MarketLinTSTrainer_MarketBandit_00001,RUNNING,192.168.1.149:44560,3,0.557334,300,427
MarketLinTSTrainer_MarketBandit_00002,RUNNING,192.168.1.149:70712,4,0.745704,400,560.065


Result for MarketLinTSTrainer_MarketBandit_00000:
  custom_metrics: {}
  date: 2020-06-20_11-45-13
  done: false
  episode_len_mean: 91.0
  episode_reward_max: 757.41
  episode_reward_mean: 550.0306896551724
  episode_reward_min: 348.20000000000005
  episodes_this_iter: 1
  episodes_total: 29
  experiment_id: ee3f8118edfb4478a10227cde029b77d
  experiment_tag: '0'
  grad_time_ms: 0.384
  hostname: DWAnyscaleMBP.local
  info:
    grad_time_ms: 0.384
    learner:
      cumulative_regret: 24249.0
      update_latency: 0.000331878662109375
    num_steps_sampled: 2700
    num_steps_trained: 2700
    opt_peak_throughput: 2601.764
    opt_samples: 1.0
    sample_peak_throughput: 586.075
    sample_time_ms: 1.706
    update_time_ms: 0.001
  iterations_since_restore: 27
  learner:
    cumulative_regret: 24249.0
    update_latency: 0.000331878662109375
  node_ip: 192.168.1.149
  num_healthy_workers: 0
  num_steps_sampled: 2700
  num_steps_trained: 2700
  off_policy_estimator: {}
  opt_peak_throug

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,RUNNING,192.168.1.149:44565,30,5.67124,3000,551.688
MarketLinTSTrainer_MarketBandit_00001,RUNNING,192.168.1.149:44560,29,5.46309,2900,563.895
MarketLinTSTrainer_MarketBandit_00002,RUNNING,192.168.1.149:70712,29,5.5195,2900,596.37


...

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,RUNNING,192.168.1.149:44565,54,10.5769,5400,563.437
MarketLinTSTrainer_MarketBandit_00001,RUNNING,192.168.1.149:44560,53,10.3575,5300,565.202
MarketLinTSTrainer_MarketBandit_00002,RUNNING,192.168.1.149:70712,53,10.4475,5300,603.161


Result for MarketLinTSTrainer_MarketBandit_00000:
  custom_metrics: {}
  date: 2020-06-20_11-45-23
  done: false
  episode_len_mean: 91.0
  episode_reward_max: 767.54
  episode_reward_mean: 571.4274698795182
  episode_reward_min: 344.0300000000001
  episodes_this_iter: 1
  episodes_total: 83
  experiment_id: ee3f8118edfb4478a10227cde029b77d
  experiment_tag: '0'
  grad_time_ms: 0.391
  hostname: DWAnyscaleMBP.local
  info:
    grad_time_ms: 0.391
    learner:
      cumulative_regret: 66489.0
      update_latency: 0.00024509429931640625
    num_steps_sampled: 7600
    num_steps_trained: 7600
    opt_peak_throughput: 2555.632
    opt_samples: 1.0
    sample_peak_throughput: 627.186
    sample_time_ms: 1.594
    update_time_ms: 0.001
  iterations_since_restore: 76
  learner:
    cumulative_regret: 66489.0
    update_latency: 0.00024509429931640625
  node_ip: 192.168.1.149
  num_healthy_workers: 0
  num_steps_sampled: 7600
  num_steps_trained: 7600
  off_policy_estimator: {}
  opt_peak_thr

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,RUNNING,192.168.1.149:44565,78,15.3519,7800,572.711
MarketLinTSTrainer_MarketBandit_00001,RUNNING,192.168.1.149:44560,78,15.3278,7800,564.714
MarketLinTSTrainer_MarketBandit_00002,RUNNING,192.168.1.149:70712,78,15.4532,7800,602.129


Result for MarketLinTSTrainer_MarketBandit_00000:
  custom_metrics: {}
  date: 2020-06-20_11-45-28
  done: true
  episode_len_mean: 91.0
  episode_reward_max: 768.38
  episode_reward_mean: 582.1036
  episode_reward_min: 344.0300000000001
  episodes_this_iter: 1
  episodes_total: 109
  experiment_id: ee3f8118edfb4478a10227cde029b77d
  experiment_tag: '0'
  grad_time_ms: 0.394
  hostname: DWAnyscaleMBP.local
  info:
    grad_time_ms: 0.394
    learner:
      cumulative_regret: 86972.0
      update_latency: 0.0002739429473876953
    num_steps_sampled: 10000
    num_steps_trained: 10000
    opt_peak_throughput: 2538.464
    opt_samples: 1.0
    sample_peak_throughput: 636.861
    sample_time_ms: 1.57
    update_time_ms: 0.001
  iterations_since_restore: 100
  learner:
    cumulative_regret: 86972.0
    update_latency: 0.0002739429473876953
  node_ip: 192.168.1.149
  num_healthy_workers: 0
  num_steps_sampled: 10000
  num_steps_trained: 10000
  off_policy_estimator: {}
  opt_peak_throughput

Trial name,status,loc,iter,total time (s),ts,reward
MarketLinTSTrainer_MarketBandit_00000,TERMINATED,,100,19.8495,10000,582.104
MarketLinTSTrainer_MarketBandit_00001,TERMINATED,,100,19.8166,10000,570.436
MarketLinTSTrainer_MarketBandit_00002,TERMINATED,,100,19.995,10000,609.601


## Analyzing the results

Let's analyze the rewards and cumulative regrets of these trials.

In [28]:
df_ts = pd.DataFrame()

for key, df_trial in analysis.trial_dataframes.items():
    df_ts = df_ts.append(df_trial, ignore_index=True)
    
df_ts.head()

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_steps_trained,num_steps_sampled,sample_time_ms,grad_time_ms,update_time_ms,...,info/update_time_ms,info/opt_peak_throughput,info/sample_peak_throughput,info/opt_samples,learner/cumulative_regret,learner/update_latency,perf/cpu_util_percent,perf/ram_util_percent,info/learner/cumulative_regret,info/learner/update_latency
0,479.23,479.23,479.2300,91.0,1,100,100,1.535,0.306,0.003,...,0.003,3269.903,651.421,1.0,1056.0,0.000155,10.3,66.0,1056.0,0.000155
1,479.23,348.20,413.7150,91.0,1,200,200,1.485,0.298,0.001,...,0.001,3351.957,673.513,1.0,2288.0,0.000171,NaN,NaN,2288.0,0.000171
2,479.23,348.20,411.9100,91.0,1,300,300,1.491,0.287,0.001,...,0.001,3483.352,670.702,1.0,3181.0,0.000176,NaN,NaN,3181.0,0.000176
3,555.98,348.20,447.9275,91.0,1,400,400,1.590,0.300,0.001,...,0.001,3336.227,629.105,1.0,3896.0,0.000214,44.0,66.1,3896.0,0.000214
4,708.92,348.20,500.1260,91.0,1,500,500,1.584,0.335,0.001,...,0.001,2984.845,631.149,1.0,4768.0,0.000173,NaN,NaN,4768.0,0.000173


In [29]:
rewards = df_ts \
    .groupby("num_steps_trained")["episode_reward_mean"] \
    .aggregate(["mean", "max", "min", "std"])

rewards

,mean,max,min,std
num_steps_trained,,,,
100,365.256667,505.2600,111.2800,220.334974
200,447.176667,565.8300,361.9850,105.962061
300,459.626667,539.9700,411.9100,69.987252
400,486.991667,560.0650,447.9275,63.333816
500,510.726000,557.2660,474.7860,42.249350
...,...,...,...,...
9600,585.569600,607.3247,568.8095,19.737469
9700,585.068633,607.2699,568.1063,20.100540
9800,585.685367,607.2624,567.1203,20.239837


In [30]:
plot_line_with_stddev(rewards, x_col='num_steps_trained', y_col='mean', stddev_col='std', 
                      title='Rewards vs. Steps', x_axis_label='step', y_axis_label='reward')

([image](../../images/rllib/Market-Bandit-Rewards-vs-Steps.png))

The rewards reach what appears to be nearly optimal by 3000 steps, then shows some slow improvement beyond 8000.

In [31]:
regrets = df_ts \
    .groupby("num_steps_trained")["learner/cumulative_regret"] \
    .aggregate(["mean", "max", "min", "std"])

regrets

,mean,max,min,std
num_steps_trained,,,,
100,1142.666667,1408.0,964.0,234.344476
200,2118.666667,2288.0,1859.0,228.320681
300,3036.666667,3181.0,2776.0,226.177659
400,3868.333333,4080.0,3629.0,226.769339
500,4679.000000,4768.0,4510.0,146.427456
...,...,...,...,...
9600,82970.666667,84884.0,80568.0,2199.215618
9700,83834.333333,85769.0,81347.0,2262.211602
9800,84492.000000,86407.0,81958.0,2288.180718


In [32]:
plot_cumulative_regret(regrets)

([image](../../images/rllib/Market-Bandit-Cumulative-Regret.png))

## Evaluating the Trained Policy

Overall, how well did the trained policy perform? The results should be better than random, but less than the best case.

In [33]:
print("{:5.2f}% optimized return annualized".format(max(rewards["mean"]) / n_years))

 6.38% optimized return annualized
(pid=70758) 2020-06-20 11:47:49,074	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=70758) 2020-06-20 11:47:49,078	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=70758) /Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=70758)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=70759) 2020-06-20 11:47:49,074	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=70759) 2020-06-20 11:47:49,078	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=70759) /Users/deanwampler/anaconda3/envs/anyscale-academy/lib/python3.7/site-packag

You should see a number between 6-8%. That's better than the random action baseline of 3.75%, but no where near the best case scenario of 15.18% return. Hence, our regrets continue to grow over time...

Note that investing solely in the S&P stock index which would have produced better than 8% return over that period -- that is, if one could wait 92 years. However, investing one's entire portfolio into stocks can become quite a risky policy in the short-term, so we were exploring how to balance a portfolio given only limited information.

In any case, the contextual bandit performed well considering that it could only use *inflation* for the context of its decisions, and could only take actions once each year.

## Exercise 1

Try using a `LinUCBTrainer`-based trainer. How does the annualized return compare?

---

## Extra - Restoring from a Checkpoint

In the previous lesson, [05 Thompson Sampling](05-Thompson-Sampling.ipynb), we showed how to restore a trainer from a checkpoint, but almost "in passing". Let's use this feature again, this time with our custom trainer class `MarketLinTSTrainer`.

In [29]:
trial = analysis.trials[0]
path = trial.checkpoint.value
print(f'checkpoint_path: {path}')

checkpoint_path: /Users/paco/ray_results/MarketLinTSTrainer/MarketLinTSTrainer_MarketBandit_0_2020-06-15_10-19-40ufbjjnn4/checkpoint_100/checkpoint-100


In [30]:
trainer = MarketLinTSTrainer(market_config)  # create instance and then restore from checkpoint
trainer.restore(path)

2020-06-15 10:24:14,450	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-06-15 10:24:14,463	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/opt/anaconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
2020-06-15 10:24:14,476	INFO trainable.py:217 -- Getting current IP.
2020-06-15 10:24:14,477	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020-06-15 10:24:14,486	INFO trainable.py:217 -- Getting current IP.
2020-06-15 10:24:14,488	INFO trainable.py:423 -- Restored on 192.168.1.244 from checkpoint: /Users/paco/ray_results/MarketLinTSTrainer/MarketLinTSTrainer_MarketBandit_0_2020-06-15_10-19-40ufbjjnn4/checkpoint_100/checkpoint-100
2020-06-15 10:24:14,489	INFO trainable.py:430 -- Current state a

MarketBandit: max_inflation: 100.0, tickers: ['sp500', 't.bill', 't.bond', 'corp'], data file: /Users/paco/src/academy/ray-rllib/multi-armed-bandits/market.tsv (config: {})


Access the model, to review the distribution of arm weights

In [32]:
model = trainer.get_policy().model
means = [model.arms[i].theta.numpy() for i in range(3)]
covs = [model.arms[i].covariance.numpy() for i in range(3)]
means, covs, model.arms[0].theta.numpy()

([array([1.3029345], dtype=float32),
  array([-0.08178909], dtype=float32),
  array([0.17635827], dtype=float32)],
 [array([[6.5596405e-06]], dtype=float32),
  array([[3.9466147e-05]], dtype=float32),
  array([[2.8418384e-05]], dtype=float32)],
 array([1.3029345], dtype=float32))

A final note: when you checkpoint it will change how the training performs in this notebook, if you rerun the training! So be sure to start from scratch when doing experiments here, if that's what you intend!